In [19]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare two ensemble outputs (e.g., gauge-based GMET and NLDAS-based GMET)

from mpl_toolkits.basemap import Basemap
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime

def read_ens(out_forc_name_base, ens_num):
    for i in range(ens_num):
        ens_file = os.path.join(out_forc_name_base + '.' + str('%03d' % (i+1)) +'.nc')
        
        f=xr.open_dataset(ens_file)
        pcp = f.variables['pcp'][:]
        t_mean = f.variables['t_mean'][:]
        t_range = f.variables['t_range'][:]

        if i == 0:
            lats = f['latitude'].values[:] #shape (y,x)
            lons = f['longitude'].values[:]
            time = pd.DatetimeIndex(f['time'][:].dt.floor('D').to_pandas())
                 
            pcp_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))# create ens array 
            t_mean_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))
            t_range_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))

        pcp_ens[:,:,:,i] = pcp
        t_mean_ens[:,:,:,i] = t_mean
        t_range_ens[:,:,:,i] = t_range
       
    return time, pcp_ens, t_mean_ens, t_range_ens

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0):

    #     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)
    
    m.drawmapboundary(color='k', linewidth=1.)
    m.drawcoastlines()
    m.drawparallels(np.arange(np.floor(llcrnrlat),np.ceil(urcrnrlat),0.2),labels=[True,False,False,False],dashes=[1,1], fontsize='small') # Draw parallels (latitude lines) for values (in degrees).
    m.drawmeridians(np.arange(np.floor(llcrnrlon),np.ceil(urcrnrlon),0.2),labels=[False,False,False,True],dashes=[1,1], fontsize='small') # Draw meridians (longitude lines). Label [left, right, top, bottom]
    m.drawstates(linewidth=0.5, linestyle='solid', color='k')
#     m.drawrivers(linewidth=1.0, linestyle='solid', color='blue')
    return m

# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar.
    source: http://chris35wills.github.io/matplotlib_diverging_colorbar/
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-300, vmax=1000))    
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#======================================================================================================
# main script
root_dir='/glade/u/home/hongli/work/russian/ens_forc_wrf2'
# root_dir='/home/hongli/work/russian/ens_forc'

wrf_raw_file = os.path.join(root_dir,'scripts/step1_asc_to_nc/WestWRF_daily_20171201_20180408.nc')
result_dir = os.path.join(root_dir,'test_random')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
ens_num = 100
time_format = '%Y-%m-%d'

asc_dir = os.path.join(root_dir, 'scripts/hmet_ascii_data_1day_lead')
asc_files = [f for f in os.listdir(asc_dir) if '.asc' in f]
asc_files = sorted(asc_files)

dpi_value = 90
plot_date_start = '2017-12-02'
plot_date_end = '2018-04-08'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)

output_dir=os.path.join(root_dir, 'scripts/step9_plot_spatial_ens_random')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_filename_base = 'spatial_'
    
#======================================================================================================
# read historical wrf data
print('Read WRF data')
f_wrf = xr.open_dataset(wrf_raw_file)
wrf_lat = f_wrf['lat'].values[:] # (y, x)
wrf_lon = f_wrf['lon'].values[:] # (y, x)
wrf_prcp = f_wrf['prcp'].values[:] # (time, y, x). unit: mm/day
wrf_tmin = f_wrf['tmin'].values[:] # (time, y, x). unit: degC
wrf_tmax = f_wrf['tmax'].values[:] 
wrf_time = pd.to_datetime(f_wrf['time'].values[:])

# wrf_time = pd.to_datetime(f_wrf['time'].values[:]).strftime(time_format)
# wrf_time = np.asarray([datetime.datetime.strptime(t, time_format) for t in wrf_time])
wrf_tmean = 0.5*(wrf_tmax + wrf_tmin)
wrf_trange = wrf_tmax - wrf_tmin
f_wrf.close()

prcp_mean = np.nanmean(wrf_prcp, axis = 0)
tmean_mean = np.nanmean(wrf_tmean, axis = 0)
trange_mean = np.nanmean(wrf_trange, axis = 0)

llcrnrlon = wrf_lon[0,0]
urcrnrlon = wrf_lon[-1,-1]
llcrnrlat = wrf_lat[0,0]
urcrnrlat = wrf_lat[-1,-1]
lat_0=0.5*(llcrnrlat+urcrnrlat)
lon_0=0.5*(llcrnrlon+urcrnrlon)

#======================================================================================================
print('Plot')
# loop through all uniform tests
for test_folder in test_folders[0:1]:
    
    print(test_folder)
    test_dir = os.path.join(result_dir, test_folder)

    # read output ensemble
    wrf_out_forc_name_base = os.path.join(test_dir,'outputs', 'ens_forc')
    wrf_ens_time, wrf_pcp_ens, wrf_t_mean_ens, wrf_t_range_ens = read_ens(wrf_out_forc_name_base, ens_num)

    # define plot mask for nldas ensemble
    wrf_ens_mask_time = (wrf_ens_time>=plot_date_start_obj) & (wrf_ens_time<=plot_date_end_obj)
    
    # calculate mean and std
    pcp_ens_mean = np.nanmean(np.nanmean(wrf_pcp_ens[wrf_ens_mask_time,:,:,:], axis = 3),axis=0) 
    pcp_ens_std = np.nanmean(np.std(wrf_pcp_ens[wrf_ens_mask_time,:,:,:], axis = 3),axis=0)
    
    tmean_ens_mean = np.nanmean(np.nanmean(wrf_t_mean_ens[wrf_ens_mask_time,:,:,:], axis = 3),axis=0)
    tmean_ens_std = np.nanmean(np.std(wrf_t_mean_ens[wrf_ens_mask_time,:,:,:], axis = 3),axis=0)

    trange_ens_mean = np.nanmean(np.nanmean(wrf_t_range_ens[wrf_ens_mask_time,:,:,:], axis = 3),axis=0)
    trange_ens_std = np.nanmean(np.std(wrf_t_range_ens[wrf_ens_mask_time,:,:,:], axis = 3),axis=0)
    
    # setup plot colorbar range for the plot_date
    vmin_prcp_mean=np.nanmin([np.nanmin(prcp_mean), np.nanmin(pcp_ens_mean)])
    vmax_prcp_mean=np.nanmax([np.nanmax(prcp_mean), np.nanmax(pcp_ens_mean)])
    vmin_prcp_std=np.nanmin(pcp_ens_std)
    vmax_prcp_std=np.nanmax(pcp_ens_std)
    
    vmin_t_mean_mean=np.nanmin([np.nanmin(tmean_mean), np.nanmin(tmean_ens_mean)])
    vmax_t_mean_mean=np.nanmax([np.nanmax(tmean_mean), np.nanmax(tmean_ens_mean)])
    vmin_t_mean_std=np.nanmin(tmean_ens_std)
    vmax_t_mean_std=np.nanmax(tmean_ens_std)
    
    vmin_t_range_mean=np.nanmin([np.nanmin(trange_mean), np.nanmin(trange_ens_mean)])
    vmax_t_range_mean=np.nanmax([np.nanmax(trange_mean), np.nanmax(trange_ens_mean)])
    vmin_t_range_std=np.nanmin(trange_ens_std)
    vmax_t_range_std=np.nanmax(trange_ens_std)

    # plot
    nrow = 3 # prcp, tmean, trange
    ncol = 3 # WRF, ens mean, ens std
    fig, ax = plt.subplots(nrow, ncol)
    fig.set_figwidth(4*ncol) 
    fig.set_figheight(4*nrow)
    
    for i in range(nrow):
        for j in range(ncol):
    
            # plot Basemap
            m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax[i,j],lat_0,lon_0) # plot Basemap 
            
            # select data for each subplot
            # PCP (first row)
            if i == 0 and j == 0:
                data=prcp_mean
                cmap=plt.cm.Blues
                vmin=vmin_prcp_mean
                vmax=vmax_prcp_mean
                title_str = '(a) WRF Daily Precip'
            elif i == 0 and j == 1:
                data=pcp_ens_mean
                cmap=plt.cm.Blues
                vmin=vmin_prcp_mean
                vmax=vmax_prcp_mean
                title_str = '(b) Ens Mean of Daily Precip'
            elif i == 0 and j == 2:
                data=pcp_ens_std
                cmap=plt.cm.Blues
                vmin=vmin_prcp_std
                vmax=vmax_prcp_std
                title_str = '(c) Ens Std of Daily Precip'
    
            # T_MEAN (second row)
            elif i == 1 and j == 0:
                data=tmean_mean
                cmap=plt.cm.Reds
                vmin=vmin_t_mean_mean
                vmax=vmax_t_mean_mean
                title_str = '(d) WRF Mean Temp'
            elif i == 1 and j == 1:
                data=tmean_ens_mean
                cmap=plt.cm.Reds
                vmin=vmin_t_mean_mean
                vmax=vmax_t_mean_mean
                title_str = '(e) Ens Mean of Mean Temp'
            elif i == 1 and j == 2:
                data=tmean_ens_std
                cmap=plt.cm.Reds
                vmin=vmin_t_mean_std
                vmax=vmax_t_mean_std
                title_str = '(f) Ens Std of Mean Temp'
    
            # T_RANGE (third row)
            elif i == 2 and j == 0:
                data=trange_mean
                cmap=plt.cm.Greens
                vmin=vmin_t_range_mean
                vmax=vmax_t_range_mean
                title_str = '(g) WRF Temp Range'
            elif i == 2 and j == 1:
                data=trange_ens_mean
                cmap=plt.cm.Greens
                vmin=vmin_t_range_mean
                vmax=vmax_t_range_mean
                title_str = '(h) Ens Mean of Temp Range'
            elif i == 2 and j == 2:
                data=trange_ens_std
                cmap=plt.cm.Greens #bwr
                vmin=vmin_t_range_std
                vmax=vmax_t_range_std
                title_str = '(i) Ens Std of Temp Range'

            # plot data
            im1 = m.pcolormesh(wrf_lon,wrf_lat,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)

            # set title
            ax[i,j].set_title(title_str, fontsize='small', fontweight='semibold')

            # set colorbar
            cbar = m.colorbar(im1, location='right')
            if i == 0:
                cbar.set_label(label='(mm/day)', size='small', rotation='horizontal', labelpad=-20, y=1.04) 
            elif i >= 1:
                cbar.set_label(label='($^\circ$C)', size='small', rotation='horizontal', labelpad=-20, y=1.04)
            cbar.ax.tick_params(labelsize='small') 
    
    # save plot
    fig.tight_layout()
    output_filename = output_filename_base+test_folder+'.png'
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)
    #     plt.show()
    del wrf_pcp_ens, wrf_t_mean_ens, wrf_t_range_ens

print('Done')


Read WRF data
Plot
010percent


/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/xarray/core/dataarray.py:1658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  3: pd.Panel}
/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:43: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:49: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.


Done


In [18]:
llcrnrlon,urcrnrlon,llcrnrlat,urcrnrlat ,lat_0,lon_0

(-123.50623,
 -122.87769,
 38.90605,
 39.47912,
 39.19258499145508,
 -123.19196319580078)

In [5]:
np.shape(x), np.shape(y)

((23, 20), (23, 20))